In [27]:
pip install findspark


Note: you may need to restart the kernel to use updated packages.


In [4]:
import findspark
findspark.init()


In [5]:
import pyspark
from pyspark.sql import SparkSession

In [6]:

spark = SparkSession.builder.getOrCreate()

23/07/07 10:13:54 WARN Utils: Your hostname, Saanils-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.242 instead (on interface en0)
23/07/07 10:13:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/07 10:13:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [8]:
from pyspark.sql.functions import explode
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, col

spark.read.json('/Users/saanilkhanna/Downloads/Fetch /py/Submission/brands.json.gz').createOrReplaceTempView("brands")
spark.read.json('/Users/saanilkhanna/Downloads/Fetch /py/Submission/receipts.json.gz').createOrReplaceTempView("receipts")
spark.read.json('/Users/saanilkhanna/Downloads/Fetch /py/Submission/users.json.gz').createOrReplaceTempView("users")

df = spark.sql("""select _id.*, * from users""")
df.withColumnRenamed(df.columns[0], 'user_id_new').createOrReplaceTempView("users")

df = spark.sql("""select cpg.*, * from brands""")
df.select("$id.$oid", '*').withColumnRenamed('$oid', 'cpg_id').createOrReplaceTempView("brands")

spark.sql("""select * from receipts""").withColumn("rewardsReceiptItemList", explode("rewardsReceiptItemList")).createOrReplaceTempView("receipts")

23/07/07 10:15:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
spark.sql("""
select * from Receipts r
inner join Users u on r.userId = u.user_id_new
inner join Brands b on r.rewardsReceiptItemList.pointsPayerId = b.cpg_id;
""").dropDuplicates().withColumn('receipt_id', col('r._id.$oid'))\
  .withColumn('createDate', from_unixtime(col('createDate.$date') / 1000).cast('timestamp'))\
    .withColumn('dateScanned', from_unixtime(col('dateScanned.$date') / 1000).cast('timestamp'))\
      .withColumn('finishedDate', from_unixtime(col('finishedDate.$date') / 1000).cast('timestamp'))\
        .withColumn('modifyDate', from_unixtime(col('modifyDate.$date') / 1000).cast('timestamp'))\
          .withColumn('pointsAwardedDate', from_unixtime(col('pointsAwardedDate.$date') / 1000).cast('timestamp'))\
            .withColumn('purchaseDate', from_unixtime(col('purchaseDate.$date') / 1000).cast('timestamp'))\
              .withColumn('createdDate', from_unixtime(col('createdDate.$date') / 1000).cast('timestamp'))\
                .withColumn('lastLogin', from_unixtime(col('lastLogin.$date') / 1000).cast('timestamp'))\
                  .drop(*['_id'])\
                    .select('receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate', 'dateScanned', 'finishedDate', 'modifyDate',        'pointsAwardedDate', 'pointsEarned', 'purchaseDate', 'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent', 'userId', 'active', 'createdDate', 'role', 'signUpSource', 'state', 'cpg_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name', 'topBrand', 'receipt_id')\
                      .createOrReplaceTempView("shopping")

In [11]:
spark.sql("""SELECT * FROM shopping""").show()

+--------------------+-----------------+-----------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------+-------------------+------------------+----------------------+--------------------+----------+--------------------+------+-------------------+--------+------------+-----+--------------------+------------+--------------------+-------------------+-------------+--------------------+--------+--------------------+
|          receipt_id|bonusPointsEarned|bonusPointsEarnedReason|         createDate|        dateScanned|       finishedDate|         modifyDate|  pointsAwardedDate|pointsEarned|       purchaseDate|purchasedItemCount|rewardsReceiptItemList|rewardsReceiptStatus|totalSpent|              userId|active|        createdDate|    role|signUpSource|state|              cpg_id|     barcode|           brandCode|           category| categoryCode|                name|topBrand|          receipt_id|
+--------------------+--

In [1]:

import pandas as pd







In [19]:
import pandas as pd

df=spark.sql("""
select * from Receipts r
inner join Users u on r.userId = u.user_id_new
inner join Brands b on r.rewardsReceiptItemList.pointsPayerId = b.cpg_id;
""").dropDuplicates().withColumn('receipt_id', col('r._id.$oid'))\
  .withColumn('createDate', from_unixtime(col('createDate.$date') / 1000).cast('timestamp'))\
    .withColumn('dateScanned', from_unixtime(col('dateScanned.$date') / 1000).cast('timestamp'))\
      .withColumn('finishedDate', from_unixtime(col('finishedDate.$date') / 1000).cast('timestamp'))\
        .withColumn('modifyDate', from_unixtime(col('modifyDate.$date') / 1000).cast('timestamp'))\
          .withColumn('pointsAwardedDate', from_unixtime(col('pointsAwardedDate.$date') / 1000).cast('timestamp'))\
            .withColumn('purchaseDate', from_unixtime(col('purchaseDate.$date') / 1000).cast('timestamp'))\
              .withColumn('createdDate', from_unixtime(col('createdDate.$date') / 1000).cast('timestamp'))\
                .withColumn('lastLogin', from_unixtime(col('lastLogin.$date') / 1000).cast('timestamp'))\
                  .drop(*['_id'])\
                    .select('receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate', 'dateScanned', 'finishedDate', 'modifyDate',        'pointsAwardedDate', 'pointsEarned', 'purchaseDate', 'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent', 'userId', 'active', 'createdDate', 'role', 'signUpSource', 'state', 'cpg_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name', 'topBrand', 'receipt_id')\
                      

pandas_df = df.toPandas()

pandas_df.to_csv('Final.csv', index=False)

In [10]:

df= spark.sql("""select * from receipts""").withColumn("rewardsReceiptItemList", explode("rewardsReceiptItemList")) 

pandas_df = df.toPandas()

pandas_df.to_csv('export_receipts.csv', index=False)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "explode(rewardsReceiptItemList)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "rewardsReceiptItemList" has the type "STRUCT<barcode: STRING, brandCode: STRING, competitiveProduct: BOOLEAN, competitorRewardsGroup: STRING, deleted: BOOLEAN, description: STRING, discountedItemPrice: STRING, finalPrice: STRING, itemNumber: STRING, itemPrice: STRING, metabriteCampaignId: STRING, needsFetchReview: BOOLEAN, needsFetchReviewReason: STRING, originalFinalPrice: STRING, originalMetaBriteBarcode: STRING, originalMetaBriteDescription: STRING, originalMetaBriteItemPrice: STRING, originalMetaBriteQuantityPurchased: BIGINT, originalReceiptItemText: STRING, partnerItemId: STRING, pointsEarned: STRING, pointsNotAwardedReason: STRING, pointsPayerId: STRING, preventTargetGapPoints: BOOLEAN, priceAfterCoupon: STRING, quantityPurchased: BIGINT, rewardsGroup: STRING, rewardsProductPartnerId: S